# Random Walks

In [ ]:
import numpy as np                # Import NumPy package
import matplotlib.pyplot as plt   # Import Matplotlib package to plot graphs
plt.rcParams['figure.dpi'] = 100  # Set the figure dpi to 100 globally

# from numba import njit # Uncomment this line if you want to use Numba to speed up your code.

In [ ]:
# @njit  # Uncomment this line if you want to use Numba to speed up your code.
def walk(n_steps=10_000):
    '''
    This function produces a single random walk with a specific number of steps.

    Parameters:
    -----------
    • n_steps : The number of steps in the random walk.
   
    Returns:
    --------
    The function returns one value:
    • r: An (n_steps x 2) array containing each individual step of the random walk. 
         Each step has 2 coordinates, i.e. (x,y).
    '''
    
    r = np.zeros(shape=(n_steps, 2), dtype=np.int32)  # A 2D array of zeros to store the 
                                                      # (x,y) coordinates of each of the N steps
        
        # Q3(a) ====> Fill this in using the algorithm provided.
        
        # Later, in Q3(d) ====> implement periodic boundary conditions in this function.
        
    return r                                          # Finally, return the history of this random walk

In [ ]:
# This function has been written for you, you don't need to edit it. 
def simulate(n_walks = 50, n_steps = 10_000, bc=False, L=30):
    '''
    A function to simulate multiple random walks of some number of steps, with or without boundary conditions.
    This function has been written for you, you don't need to edit it unless you want to.

    Parameters:
    -----------
    • n_walks : The number of random walks to simulate.
    • n_steps : The number of steps in each random walk.
    • bc      : A boolean variable that decides if periodic boundary conditions in a box are enforced or not.
    • L       : The box-size is given by 2L if periodic boundary conditions are enforced.

    Returns:
    --------
    The function returns one value:
    • all_walks: An (n_walks x n_steps x 2) array containing each individual step of each random walk. 
                 The first dimension is the walk-number, the second the step-number, and the third the 
                 third the coordinate (x or y).
    '''
    
    all_walks = np.zeros((n_walks, n_steps, 2))   # Array to store data of each walk

    for w in range(n_walks):                      # For each walk,

        cmap = plt.colormaps['viridis']           # Each colour will be chosen from the `viridis` cmap, with
        d_color = 1/n_walks                       # "spacing" between colours of 1/n_walks, so the full map is used

        rw = walk(n_steps=n_steps, bc=bc, L=L)    # Simulate a random walk ith required parameters
        all_walks[w] = rw                         # Store the simulated walk in `all_walks`
        
        # Plot the random walk
        plt.plot(rw[:,0], rw[:,1], marker='o', ms=0.01, color=cmap(w*d_color), alpha=1)
    
    plt.ylabel(r"$y$")
    plt.xlabel(r"$x$")

    # Plot the origin of the walks. The zorder variable decides which plot is plotted after which one,
    # and makes sure that the scatter point is visible and not plotted _underneath_ the other plots.
    plt.scatter(0, 0, color='firebrick', s=10, zorder=n_walks+1)        
    plt.gca().set_aspect('equal') # Set the aspect ratio of the axis to equal, so a square looks like a square
    
    return all_walks # Return the final array of all the walks

In [ ]:
%%time
# This cell stores all the walk data for 50 walks of 10_000 steps each in `all_walks`. It should take about
# 10 seconds to run, if no errors have been made.

all_walks = simulate() # Q3(b) ====> Run this.

In [ ]:
def plot_evolution(all_walks, plot_L=200):
    '''
    A function to plot the evolution of generated multiple random walks.
    This function has been written for you, you don't need to edit it unless you want to.

    Parameters:
    -----------
    • all_walks : Position data of a group of random walks, as a 3D array of dimension (n_walks x n_steps x 2).
    • plot_L    : x and y limits of the resulting plot are taken to be -plot_L and plot_L respectively.

    Returns:
    --------
    NoneType (A plot).

    '''
    
    n_walks = len(all_walks)    # Number of walks
    n_steps = len(all_walks[0]) # Number of steps per walk
    
    ncols=5                     # Number of intervals. Plotted as columns in a figure below
    
    del_steps = n_steps//ncols  # Steps per interval. The double-slash indicates "integer-division" eg 5//3=1.

    # Define a figure and a set of axes of 1 row and 5 columns (default) and some figure size
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(4*ncols,4))

    cmap = plt.colormaps['viridis'] # Set the colour map
    d_color = 1/n_walks             # and the interval between colours, as before, so that the full map is used

    axes[0].set_ylabel(r"$y$")
    
    for i in range(ncols):          # For each interval,

        axes[i].scatter(0, 0, color='firebrick', s=10, zorder=n_walks+1)  # Plot the start-point of the walk

        # "Clip" the walk so that the first `(i+1)*del_steps` steps is chosen for each walk.
        # This will look confusing for those of you not familiar with NumPy, but you will get used to it by the
        # end of the course! You don't _need_ to write code like this, but it makes things faster.
        clipped_walks = all_walks[:,:(i+1)*del_steps,:]                   
        
        # For each "clipped walk"
        for j in range(n_walks):
            # Plot the walk in the appropriate column of the figure (i.e. in axes[i]) with a different colour
            axes[i].scatter(clipped_walks[j,:,0], clipped_walks[j,:,1],  s=0.1, color=cmap(j*d_color), alpha=1, zorder=i)
        
        axes[i].set_xlim(-plot_L,plot_L) # Set the axes limits based on the `plot_L` parameter
        axes[i].set_ylim(-plot_L,plot_L) # The result is always square
        axes[i].set_aspect('equal')      # Set the axes aspect ratio to be square
        axes[i].set_xlabel(r"$x$")       # Set the x-label for each plot
        
    plt.tight_layout()

In [ ]:
%%time
# This cell plots the evolution of the walks stored in the `all_walks` variable created above.

plot_evolution(all_walks) # Q3(b) ====> Run this as well.

In [ ]:
def get_Rsq(walk):
    '''
    A function get how the R-squared values change over the evolution of a single random walk.

    Parameters:
    -----------
    • walk  : Position data of a single of random walk, as a 2D array of dimension (n_steps x 2).

    Returns:
    --------
    The function returns a single value:
    • Rsq  : Array of same length as `walk`, with R-squared values at everu point between the 
             start and the end of the walk.

    '''  
    
    Rsq = np.zeros(len(walk))
    
    # Q3(c) =====> Fill in the `Rsq` array with the value of R-squared at every step in `walk`.
        
    return Rsq

In [ ]:
# Finding R-squared values for all walks

all_Rsq = np.zeros((len(all_walks), len(all_walks[0])))  # Define array to store R-squared data for each walk

# Q3(c) ====> Fill this in. For each walk, get and store the R-squared data in the all_Rsq array.


In [ ]:
# Q3(c) ====> Fill this in. 
#             Plot the averaged R-squared data (averaged over _walks_) as a function of the number of steps.

In [ ]:
# Q3(d) ====> Fill this in.
#             Implement periodic boundary conditions, and repeat all the results in Q3(a)--Q3(c).